In [2]:
#Step 0. 필요한 모듈과 라이브러리를 로딩합니다.
import sys # 시스템
import os  # 시스템

import pandas as pd  # 판다스 : 데이터분석 라이브러리
import numpy as np   # 넘파이 : 숫자, 행렬 데이터 라이브러리

import chromedriver_autoinstaller
import selenium
from selenium.webdriver.common.by import By
from selenium import webdriver   # 웹 브라우저 자동화
import time    # 서버와 통신할 때 중간중간 시간 지연. 보통은 1초
from tqdm import tqdm   # for문 돌릴 때 진행상황을 %게이지로 알려준다.

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

In [3]:
# 크롬 브라우저 (위키독스) 띄우기
chromedriver_autoinstaller.install(cwd=True)
driver = webdriver.Chrome()   # driver : 크롬 브라우저
# driver : 크롬 브라우저

driver.get('https://wikidocs.net/')
time.sleep(2)    # 2초간 정지 

In [4]:
# step1. page1 책 제목, url 크롤링 
# step2. 첫번째 책 url 들어가서 데이터 크롤링 
# step3. 모든 책(page1, 20개) url 반복문으로 들어가서 데이터 크롤링 

# step1. page1 책 제목, url 크롤링 

### 책 제목들 추출

In [5]:
# 첫번째 책 제목 추출
selector = ".book-subject"
elements = driver.find_elements(By.CSS_SELECTOR, selector)
elements[0].text

'점프 투 파이썬'

In [6]:
# 전체 책 제목 추출
title_list = []

for title in elements:
    title_list.append(title.text)
    # print(title.text)

print(len(title_list))   # page 당 책 개수는 20개 
title_list

20


['점프 투 파이썬',
 '딥 러닝을 이용한 자연어 처리 입문',
 '딥 러닝 파이토치 교과서 - 입문부터 LLM 파인튜닝까지',
 '파이썬으로 배우는 알고리즘 트레이딩 (개정판-2쇄)',
 '점프 투 자바',
 '<랭체인LangChain 노트> - LangChain 한국어 튜토리얼🇰🇷',
 '[ 문과생도 할 수 있는 ] 파이썬 업무 자동화 일잘러 되기 + 챗GPT',
 '주식 시장을 이기는 마법의 자동매매',
 '왕초보를 위한 Python: 쉽게 풀어 쓴 기초 문법과 실습',
 '초보자를 위한 파이썬 300제',
 'PyQt5 Tutorial - 파이썬으로 만드는 나만의 GUI 프로그램',
 '사장님 몰래 하는 파이썬 업무자동화(부제 : 들키면 일 많아짐)',
 '[Python 완전정복 시리즈] 2편 : Pandas DataFrame 완전정복',
 '점프 투 장고',
 '파이썬을 이용한 비트코인 자동매매 (개정판)',
 '위키독스',
 'C++ 이야기(A Story of C++)',
 'IT 기술 노트',
 'Must Learning with R (개정판)',
 '라즈베리 파이 문서(2021.7 이전)']

### 책 url 수집

In [7]:
# 첫번째 책 url 확인
elements[0].get_attribute('href')

'https://wikidocs.net/book/1'

In [8]:
# 전체 url 수집 후 리스트로 저장
url_list = []

for element in elements:
    url = element.get_attribute('href')
    # print(element.get_attribute('href'))
    url_list.append(url)

print(len(url_list))
url_list

20


['https://wikidocs.net/book/1',
 'https://wikidocs.net/book/2155',
 'https://wikidocs.net/book/2788',
 'https://wikidocs.net/book/110',
 'https://wikidocs.net/book/31',
 'https://wikidocs.net/book/14314',
 'https://wikidocs.net/book/8581',
 'https://wikidocs.net/book/1050',
 'https://wikidocs.net/book/2',
 'https://wikidocs.net/book/922',
 'https://wikidocs.net/book/2165',
 'https://wikidocs.net/book/6353',
 'https://wikidocs.net/book/7188',
 'https://wikidocs.net/book/4223',
 'https://wikidocs.net/book/1665',
 'https://wikidocs.net/book/20',
 'https://wikidocs.net/book/2380',
 'https://wikidocs.net/book/2184',
 'https://wikidocs.net/book/4315',
 'https://wikidocs.net/book/483']

In [9]:
# 데이터프레임으로 저장
df = pd.DataFrame({"title":title_list, "url":url_list})
df.to_csv("wikidocs_page1.csv", encoding='utf-8-sig', index=False)

# step 2. 첫번째 책 url 들어가서 데이터 크롤링 

In [10]:
df = pd.read_csv("wikidocs_page1.csv")
df.head(3)

,title,url
0,점프 투 파이썬,https://wikidocs.net/book/1
1,딥 러닝을 이용한 자연어 처리 입문,https://wikidocs.net/book/2155
2,딥 러닝 파이토치 교과서 - 입문부터 LLM 파인튜닝까지,https://wikidocs.net/book/2788


In [11]:
chromedriver_autoinstaller.install(cwd=True)
driver = webdriver.Chrome()   # driver : 크롬 브라우저
# driver : 크롬 브라우저

# 첫번째 책 url 접속
i = 0
url = df['url'][i] 
driver.get(url)    
time.sleep(2)      

In [12]:
target_info = {}

In [13]:
# 지은이 
selector = "#load_content > div.book-info.row.clearfix > div:nth-child(2) > div:nth-child(1) > a"
author = driver.find_element(By.CSS_SELECTOR, selector).text
target_info['author'] = author

In [14]:
# 최종 편집일시
selector = "#load_content > div.book-info.row.clearfix > div:nth-child(2) > div:nth-child(2)"
edit_time = driver.find_element(By.CSS_SELECTOR, selector).text
target_info['edit_time'] = edit_time

In [15]:
# 판매가
selector = "#load_content > div.book-info.row.clearfix > div:nth-child(2) > div:nth-child(4)"
price = driver.find_element(By.CSS_SELECTOR, selector).text
target_info['price'] = price

In [16]:
# 책 소개글
selector = "#load_content > div.page-content > p"
elements = driver.find_elements(By.CSS_SELECTOR, selector)
# elements[0].text

overview_str = ''  #이거 왜쓰는지 의미 파악하기✨
for element in elements:
    overview_str = overview_str + element.text + '\n'    # '\n' : 줄바꿈(엔터)
    print(element.text)
# print(overview_str)
target_info['overview_str'] = overview_str

이 책은 파이썬이란 언어를 처음 접해보는 독자들과 프로그래밍을 한 번도 해 본적이 없는 사람들을 대상으로 한다. 프로그래밍을 할 때 사용되는 전문적인 용어들을 알기 쉽게 풀어서 쓰려고 노력하였으며, 파이썬이란 언어의 개별적인 특성만을 강조하지 않고 프로그래밍 전반에 관한 사항을 파이썬이란 언어를 통해 알 수 있도록 알기 쉽게 설명하였다.
파이썬에 대한 기본적인 지식을 알고 있는 사람이라도 이 책은 파이썬 프로그래밍에 대한 흥미를 가질 수 있는 좋은 안내서가 될 것이다. 이 책의 목표는 독자가 파이썬을 통해 프로그래밍에 대한 전반적인 이해를 갖게하는 것이며, 또 파이썬이라는 도구를 이용하여 원하는 프로그램을 쉽고 재미있게 만들 수 있게 하는 것이다.
점프 투 파이썬 개정 2판이 종이책으로 출간되었습니다. (2023년 6월 15일)


In [17]:
target_info

{'author': '박응용',
 'edit_time': '최종 편집일시 : 2026년 1월 29일 12:54 오후',
 'price': 'e-book 판매가 : 8,000원 (구매하기)',
 'overview_str': '이 책은 파이썬이란 언어를 처음 접해보는 독자들과 프로그래밍을 한 번도 해 본적이 없는 사람들을 대상으로 한다. 프로그래밍을 할 때 사용되는 전문적인 용어들을 알기 쉽게 풀어서 쓰려고 노력하였으며, 파이썬이란 언어의 개별적인 특성만을 강조하지 않고 프로그래밍 전반에 관한 사항을 파이썬이란 언어를 통해 알 수 있도록 알기 쉽게 설명하였다.\n파이썬에 대한 기본적인 지식을 알고 있는 사람이라도 이 책은 파이썬 프로그래밍에 대한 흥미를 가질 수 있는 좋은 안내서가 될 것이다. 이 책의 목표는 독자가 파이썬을 통해 프로그래밍에 대한 전반적인 이해를 갖게하는 것이며, 또 파이썬이라는 도구를 이용하여 원하는 프로그램을 쉽고 재미있게 만들 수 있게 하는 것이다.\n점프 투 파이썬 개정 2판이 종이책으로 출간되었습니다. (2023년 6월 15일)\n'}

# step3. 모든 책(page1, 20개) url 반복문으로 들어가서 데이터 크롤링 

In [18]:
chromedriver_autoinstaller.install(cwd=True)  #✨ 추가  👉 “내 크롬 버전에 맞는 드라이버를, 지금 폴더에 자동으로 깔아줘”
# driver : 크롬 브라우저

# # 첫번째 책 url 접속
# i = 0

total_dict = {}   # 총 딕셔너리 추가✨

# 전체 책 url 접속 
for i in range(0, 10): # ✨ 포문만,try 추가  i = 0(생략처리함)
    try:
        driver = webdriver.Chrome()   #✨추가 driver : 크롬 브라우저
        url = df['url'][i] 
        driver.get(url)    
        time.sleep(2)     
        
        target_info = {}   # ✨ 딕셔너리 담아둔거 

        # 책제목
        selector = ".page-subject-text"
        title = driver.find_element(By.CSS_SELECTOR, selector).text
        target_info['title'] = title
        
        # 지은이 
        selector = "#load_content > div.book-info.row.clearfix > div:nth-child(2) > div:nth-child(1) > a"
        author = driver.find_element(By.CSS_SELECTOR, selector).text
        target_info['author'] = author
        
        # 최종 편집일시
        selector = "#load_content > div.book-info.row.clearfix > div:nth-child(2) > div:nth-child(2)"
        edit_time = driver.find_element(By.CSS_SELECTOR, selector).text
        target_info['edit_time'] = edit_time
        
        # 판매가
        selector = "#load_content > div.book-info.row.clearfix > div:nth-child(2) > div:nth-child(4)"
        price = driver.find_element(By.CSS_SELECTOR, selector).text
        target_info['price'] = price
        

        # 책 소개글
        
        
        # 책 소개글
        selector = "#load_content > div.page-content > p"
        elements = driver.find_elements(By.CSS_SELECTOR, selector)
        # elements[0].text
        overview_str = ''
        for element in elements:
            overview_str = overview_str + element.text + '\n'    # '\n' : 줄바꿈(엔터)
            # print(element.text)
        target_info['overview_str'] = overview_str
        
        #위에 까지 다 그냥 가져온거 만든 거 아래 부터 공식처럼 추가
        print(target_info)
        total_dict[i] = target_info
        print(i+1, f"번째 책 '{title}' 내용이 수집되었습니다", "\n")
        
        driver.close()

    except:
        print(i+1, f"번째 책 '{title}' 에서 에러가 발생")
        continue

{'title': '점프 투 파이썬', 'author': '박응용', 'edit_time': '최종 편집일시 : 2026년 1월 29일 12:54 오후', 'price': 'e-book 판매가 : 8,000원 (구매하기)', 'overview_str': '이 책은 파이썬이란 언어를 처음 접해보는 독자들과 프로그래밍을 한 번도 해 본적이 없는 사람들을 대상으로 한다. 프로그래밍을 할 때 사용되는 전문적인 용어들을 알기 쉽게 풀어서 쓰려고 노력하였으며, 파이썬이란 언어의 개별적인 특성만을 강조하지 않고 프로그래밍 전반에 관한 사항을 파이썬이란 언어를 통해 알 수 있도록 알기 쉽게 설명하였다.\n파이썬에 대한 기본적인 지식을 알고 있는 사람이라도 이 책은 파이썬 프로그래밍에 대한 흥미를 가질 수 있는 좋은 안내서가 될 것이다. 이 책의 목표는 독자가 파이썬을 통해 프로그래밍에 대한 전반적인 이해를 갖게하는 것이며, 또 파이썬이라는 도구를 이용하여 원하는 프로그램을 쉽고 재미있게 만들 수 있게 하는 것이다.\n점프 투 파이썬 개정 2판이 종이책으로 출간되었습니다. (2023년 6월 15일)\n'}
1 번째 책 '점프 투 파이썬' 내용이 수집되었습니다 

{'title': '딥 러닝을 이용한 자연어 처리 입문', 'author': '브라이스 유', 'edit_time': '최종 편집일시 : 2026년 1월 9일 5:09 오후', 'price': 'e-book 판매가 : 33,000원 (구매하기)', 'overview_str': '26년 1월 기준: 누적 조회수: 1,950만 돌파 베스트셀러\n많은 분들의 피드백으로 수 년간 보완된 현업 연구원들이 작성한 딥 러닝 자연어 처리 교재 입문서입니다.\nQ) 입문자도 공부 가능한가요?\nA) 이 책은 애초 AI를 아예 처음 공부하는 입문자가 타겟입니다. 파이썬을 어느 정도 할 줄 아신다면 AI 공부를 할 수 있습니다.\nQ) 이 책은 혹시 무료로 공부 가능한가요?\nA) 거의 90% 이상의 내용을 현재 무료로 공개했습니다. 그

In [19]:
total_dict

{0: {'title': '점프 투 파이썬',
  'author': '박응용',
  'edit_time': '최종 편집일시 : 2026년 1월 29일 12:54 오후',
  'price': 'e-book 판매가 : 8,000원 (구매하기)',
  'overview_str': '이 책은 파이썬이란 언어를 처음 접해보는 독자들과 프로그래밍을 한 번도 해 본적이 없는 사람들을 대상으로 한다. 프로그래밍을 할 때 사용되는 전문적인 용어들을 알기 쉽게 풀어서 쓰려고 노력하였으며, 파이썬이란 언어의 개별적인 특성만을 강조하지 않고 프로그래밍 전반에 관한 사항을 파이썬이란 언어를 통해 알 수 있도록 알기 쉽게 설명하였다.\n파이썬에 대한 기본적인 지식을 알고 있는 사람이라도 이 책은 파이썬 프로그래밍에 대한 흥미를 가질 수 있는 좋은 안내서가 될 것이다. 이 책의 목표는 독자가 파이썬을 통해 프로그래밍에 대한 전반적인 이해를 갖게하는 것이며, 또 파이썬이라는 도구를 이용하여 원하는 프로그램을 쉽고 재미있게 만들 수 있게 하는 것이다.\n점프 투 파이썬 개정 2판이 종이책으로 출간되었습니다. (2023년 6월 15일)\n'},
 1: {'title': '딥 러닝을 이용한 자연어 처리 입문',
  'author': '브라이스 유',
  'edit_time': '최종 편집일시 : 2026년 1월 9일 5:09 오후',
  'price': 'e-book 판매가 : 33,000원 (구매하기)',
  'overview_str': '26년 1월 기준: 누적 조회수: 1,950만 돌파 베스트셀러\n많은 분들의 피드백으로 수 년간 보완된 현업 연구원들이 작성한 딥 러닝 자연어 처리 교재 입문서입니다.\nQ) 입문자도 공부 가능한가요?\nA) 이 책은 애초 AI를 아예 처음 공부하는 입문자가 타겟입니다. 파이썬을 어느 정도 할 줄 아신다면 AI 공부를 할 수 있습니다.\nQ) 이 책은 혹시 무료로 공부 가능한가요?\nA) 거의 90% 이상의 내용을 현재 무료로 공개했습니다. 그러니 무료로 

In [20]:
df_final = pd.DataFrame.from_dict(total_dict, 'index')
df_final

,title,author,edit_time,price,overview_str
0,점프 투 파이썬,박응용,최종 편집일시 : 2026년 1월 29일 12:54 오후,"e-book 판매가 : 8,000원 (구매하기)",이 책은 파이썬이란 언어를 처음 접해보는 독자들과 프로그래밍을 한 번도 해 본적이 ...
1,딥 러닝을 이용한 자연어 처리 입문,브라이스 유,최종 편집일시 : 2026년 1월 9일 5:09 오후,"e-book 판매가 : 33,000원 (구매하기)","26년 1월 기준: 누적 조회수: 1,950만 돌파 베스트셀러\n많은 분들의 피드백..."
2,딥 러닝 파이토치 교과서 - 입문부터 LLM 파인튜닝까지,브라이스 유,최종 편집일시 : 2026년 1월 9일 5:09 오후,"e-book 판매가 : 33,000원 (구매하기)",26년 1월 기준: 누적 조회수: 490만 돌파 베스트셀러\n딥 러닝 프레임워크 P...
3,파이썬으로 배우는 알고리즘 트레이딩 (개정판-2쇄),조대표,최종 편집일시 : 2023년 5월 30일 1:48 오전,"4,794 명이 추천",파이스탁 YouTube\n리디북스 Ebook 구매 바로가기\n
4,점프 투 자바,박응용,최종 편집일시 : 2026년 1월 16일 2:38 오후,"e-book 판매가 : 8,000원 (구매하기)",이 책은 프로그래머를 꿈꾸며 자바 입문서를 찾는 사람들을 위한 책이다. 이 책은 자...
5,<랭체인LangChain 노트> - LangChain 한국어 튜토리얼🇰🇷,테디노트,최종 편집일시 : 2025년 4월 30일 8:06 오후,"3,697 명이 추천","추천은 공유할 수 있는 무료 전자책을 집필하는데 정말 큰 힘이 됩니다. ""추천"" 한..."
6,[ 문과생도 할 수 있는 ] 파이썬 업무 자동화 일잘러 되기 + 챗GPT,메이허,최종 편집일시 : 2024년 12월 22일 3:40 오후,"1,895 명이 추천","""파이썬 업무 자동화 일잘러 되기 + 챗GPT""는 실무 경험을 토대로, 파이썬의 여..."
7,주식 시장을 이기는 마법의 자동매매,엑슬론,최종 편집일시 : 2023년 12월 18일 10:55 오전,"1,316 명이 추천",100세 시대를 맞아 소액투자에 적합한 자동매매에 대한 종합 지식을 배우고 무료로 ...
8,왕초보를 위한 Python: 쉽게 풀어 쓴 기초 문법과 실습,전뇌해커,최종 편집일시 : 2026년 1월 11일 5:39 오후,"e-book 판매가 : 14,000원 (구매하기)","프로그래밍을 한 번도 해본 적이 없는 분들을 위해, 파이썬을 통해 처음으로 프로그래..."
9,초보자를 위한 파이썬 300제,조대표,최종 편집일시 : 2024년 6월 23일 6:45 오후,761 명이 추천,파이썬 기초 문법을 배웠다고 해서 누구나 프로그래밍에 익숙해지는 것은 아닙니다. 프...


# step4. price, like 예외처리 하기

In [21]:
# price나 like가 없는 것은 각각 0으로 채워넣기

In [28]:
chromedriver_autoinstaller.install(cwd=True)  #✨ 추가  👉 “내 크롬 버전에 맞는 드라이버를, 지금 폴더에 자동으로 깔아줘”
# driver : 크롬 브라우저

# # 첫번째 책 url 접속
# i = 0

total_dict = {}   # 총 딕셔너리 추가✨

# 전체 책 url 접속 
for i in range(0, 10): # ✨ 포문만,try 추가  i = 0(생략처리함)
    try:
        driver = webdriver.Chrome()   #✨추가 driver : 크롬 브라우저
        url = df['url'][i] 
        driver.get(url)    
        time.sleep(2)     
        
        target_info = {}   # ✨ 딕셔너리 담아둔거 

        # 책제목
        selector = ".page-subject-text"
        title = driver.find_element(By.CSS_SELECTOR, selector).text
        target_info['title'] = title
        print(title)
        
        # 지은이 
        selector = "#load_content > div.book-info.row.clearfix > div:nth-child(2) > div:nth-child(1) > a"
        author = driver.find_element(By.CSS_SELECTOR, selector).text
        target_info['author'] = author
        print(author)
        
        # 최종 편집일시
        selector = "#load_content > div.book-info.row.clearfix > div:nth-child(2) > div:nth-child(2)"
        edit_time = driver.find_element(By.CSS_SELECTOR, selector).text
        target_info['edit_time'] = edit_time
        print(edit_time)
        
        # 판매가
        selector = "#load_content > div.book-info.row.clearfix > div:nth-child(2) > div"
        elements = driver.find_elements(By.CSS_SELECTOR, selector)
        if len(elements) == 4:   # (EBOOK) PRICE가 없을 때
            price = 0    # "값이 없습니다"
        else:                    # (EBOOK) PRICE가 있을 때
            selector = "#load_content > div.book-info.row.clearfix > div:nth-child(2) > div:nth-child(4)"
            price = driver.find_element(By.CSS_SELECTOR, selector).text
        target_info['price'] = price
        print(price)

        # 추천수
        selector = ".btn.btn-default.btn-xs strong"
        up = driver.find_element(By.CSS_SELECTOR, selector).text   # "7,675", "23"
        if ',' in up:
            up = int(up.replace(',', ''))
        else:
            up = int(up)
        target_info['up'] = up
        print(up)
        
        # 책 소개글
        selector = "#load_content > div.page-content > p"
        elements = driver.find_elements(By.CSS_SELECTOR, selector)
        # elements[0].text
        
        overview_str = ''
        for element in elements:
            overview_str = overview_str + element.text + '\n'    # '\n' : 줄바꿈(엔터)
            # print(element.text)
        target_info['overview_str'] = overview_str
        
        #위에 까지 다 그냥 가져온거 만든 거 아래 부터 공식처럼 추가
        print(target_info)
        total_dict[i] = target_info
        print(i+1, f"번째 책 '{title}' 내용이 수집되었습니다", "\n")
        
        driver.close()

    except:
        print(i+1, f"번째 책 '{title}' 에서 에러가 발생")
        continue

점프 투 파이썬
박응용
최종 편집일시 : 2026년 1월 29일 12:54 오후
e-book 판매가 : 8,000원 (구매하기)
7676
{'title': '점프 투 파이썬', 'author': '박응용', 'edit_time': '최종 편집일시 : 2026년 1월 29일 12:54 오후', 'price': 'e-book 판매가 : 8,000원 (구매하기)', 'up': 7676, 'overview_str': '이 책은 파이썬이란 언어를 처음 접해보는 독자들과 프로그래밍을 한 번도 해 본적이 없는 사람들을 대상으로 한다. 프로그래밍을 할 때 사용되는 전문적인 용어들을 알기 쉽게 풀어서 쓰려고 노력하였으며, 파이썬이란 언어의 개별적인 특성만을 강조하지 않고 프로그래밍 전반에 관한 사항을 파이썬이란 언어를 통해 알 수 있도록 알기 쉽게 설명하였다.\n파이썬에 대한 기본적인 지식을 알고 있는 사람이라도 이 책은 파이썬 프로그래밍에 대한 흥미를 가질 수 있는 좋은 안내서가 될 것이다. 이 책의 목표는 독자가 파이썬을 통해 프로그래밍에 대한 전반적인 이해를 갖게하는 것이며, 또 파이썬이라는 도구를 이용하여 원하는 프로그램을 쉽고 재미있게 만들 수 있게 하는 것이다.\n점프 투 파이썬 개정 2판이 종이책으로 출간되었습니다. (2023년 6월 15일)\n'}
1 번째 책 '점프 투 파이썬' 내용이 수집되었습니다 

딥 러닝을 이용한 자연어 처리 입문
브라이스 유
최종 편집일시 : 2026년 1월 9일 5:09 오후
e-book 판매가 : 33,000원 (구매하기)
6785
{'title': '딥 러닝을 이용한 자연어 처리 입문', 'author': '브라이스 유', 'edit_time': '최종 편집일시 : 2026년 1월 9일 5:09 오후', 'price': 'e-book 판매가 : 33,000원 (구매하기)', 'up': 6785, 'overview_str': '26년 1월 기준: 누적 조회수: 1,950만 돌파 베스트셀러\n많은 분들의 피드백으로 수 년간 보완된 현

In [ ]:
#excpet: price=0=으로 하고 하니 데이터프레임 만들었을때 0으로 채워져있긴 하더라구요

In [24]:
df_final = pd.DataFrame.from_dict(total_dict, 'index')
df_final

,title,author,edit_time,price,up,overview_str
0,점프 투 파이썬,박응용,최종 편집일시 : 2026년 1월 29일 12:54 오후,"e-book 판매가 : 8,000원 (구매하기)",7676,이 책은 파이썬이란 언어를 처음 접해보는 독자들과 프로그래밍을 한 번도 해 본적이 ...
1,딥 러닝을 이용한 자연어 처리 입문,브라이스 유,최종 편집일시 : 2026년 1월 9일 5:09 오후,"e-book 판매가 : 33,000원 (구매하기)",6785,"26년 1월 기준: 누적 조회수: 1,950만 돌파 베스트셀러\n많은 분들의 피드백..."
2,딥 러닝 파이토치 교과서 - 입문부터 LLM 파인튜닝까지,브라이스 유,최종 편집일시 : 2026년 1월 9일 5:09 오후,"e-book 판매가 : 33,000원 (구매하기)",5353,26년 1월 기준: 누적 조회수: 490만 돌파 베스트셀러\n딥 러닝 프레임워크 P...
3,파이썬으로 배우는 알고리즘 트레이딩 (개정판-2쇄),조대표,최종 편집일시 : 2023년 5월 30일 1:48 오전,0,4794,파이스탁 YouTube\n리디북스 Ebook 구매 바로가기\n
4,점프 투 자바,박응용,최종 편집일시 : 2026년 1월 16일 2:38 오후,"e-book 판매가 : 8,000원 (구매하기)",4656,이 책은 프로그래머를 꿈꾸며 자바 입문서를 찾는 사람들을 위한 책이다. 이 책은 자...
5,<랭체인LangChain 노트> - LangChain 한국어 튜토리얼🇰🇷,테디노트,최종 편집일시 : 2025년 4월 30일 8:06 오후,0,3697,"추천은 공유할 수 있는 무료 전자책을 집필하는데 정말 큰 힘이 됩니다. ""추천"" 한..."
6,[ 문과생도 할 수 있는 ] 파이썬 업무 자동화 일잘러 되기 + 챗GPT,메이허,최종 편집일시 : 2024년 12월 22일 3:40 오후,0,1895,"""파이썬 업무 자동화 일잘러 되기 + 챗GPT""는 실무 경험을 토대로, 파이썬의 여..."
7,주식 시장을 이기는 마법의 자동매매,엑슬론,최종 편집일시 : 2023년 12월 18일 10:55 오전,0,1316,100세 시대를 맞아 소액투자에 적합한 자동매매에 대한 종합 지식을 배우고 무료로 ...
8,왕초보를 위한 Python: 쉽게 풀어 쓴 기초 문법과 실습,전뇌해커,최종 편집일시 : 2026년 1월 11일 5:39 오후,"e-book 판매가 : 14,000원 (구매하기)",1153,"프로그래밍을 한 번도 해본 적이 없는 분들을 위해, 파이썬을 통해 처음으로 프로그래..."
9,초보자를 위한 파이썬 300제,조대표,최종 편집일시 : 2024년 6월 23일 6:45 오후,0,761,파이썬 기초 문법을 배웠다고 해서 누구나 프로그래밍에 익숙해지는 것은 아닙니다. 프...


# 쌤한테 받은 코드에서 지피티한테 물어서 쉽게 할 수 있는 방법 물어보기

In [38]:
#✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨✨
chromedriver_autoinstaller.install(cwd=True)  #✨ 추가  👉 “내 크롬 버전에 맞는 드라이버를, 지금 폴더에 자동으로 깔아줘”
# driver : 크롬 브라우저

# # 첫번째 책 url 접속
# i = 0

total_dict = {}   # 총 딕셔너리 추가✨

# 전체 책 url 접속 
for i in range(0, 5): # ✨ 포문만,try 추가  i = 0(생략처리함)
    try:
        driver = webdriver.Chrome()   #✨추가 driver : 크롬 브라우저
        url = df['url'][i] 
        driver.get(url)    
        time.sleep(2)     
        
        target_info = {}   # ✨ 딕셔너리 담아둔거 

        # 책제목
        selector = ".page-subject-text"
        title = driver.find_element(By.CSS_SELECTOR, selector).text
        target_info['title'] = title
        print(title)
        
        # 지은이 
        selector = "#load_content > div.book-info.row.clearfix > div:nth-child(2) > div:nth-child(1) > a"
        author = driver.find_element(By.CSS_SELECTOR, selector).text
        target_info['author'] = author
        print(author)
        
        # 최종 편집일시
        selector = "#load_content > div.book-info.row.clearfix > div:nth-child(2) > div:nth-child(2)"
        edit_time = driver.find_element(By.CSS_SELECTOR, selector).text
        target_info['edit_time'] = edit_time
        print(edit_time)
        
        # 판매가
        try:
            selector = "#load_content > div.book-info.row.clearfix > div:nth-child(2) > div:nth-child(4)"
            price = driver.find_element(By.CSS_SELECTOR, selector).text

        # 추천 문구면 0으로 처리
            if "추천" in price:
                price = 0

        except:
            price = 0

        target_info['price'] = price
        print(price)

        # 추천수
        try:
            selector = ".btn.btn-default.btn-xs strong"
            up = driver.find_element(By.CSS_SELECTOR, selector).text

            # 콤마 제거 후 숫자로 변환
            up = int(up.replace(',', ''))

        except:
            up = 0

        target_info['up'] = up
        print(up)
        
        # 책 소개글
        selector = "#load_content > div.page-content > p"
        elements = driver.find_elements(By.CSS_SELECTOR, selector)
        # elements[0].text
        
        overview_str = ''
        for element in elements:
            overview_str = overview_str + element.text + '\n'    # '\n' : 줄바꿈(엔터)
            # print(element.text)
        target_info['overview_str'] = overview_str
        
        #위에 까지 다 그냥 가져온거 만든 거 아래 부터 공식처럼 추가
        print(target_info)
        total_dict[i] = target_info
        print(i+1, f"번째 책 '{title}' 내용이 수집되었습니다", "\n")
        
        driver.close()

    except:
        print(i+1, f"번째 책 '{title}' 에서 에러가 발생")
        continue

점프 투 파이썬
박응용
최종 편집일시 : 2026년 1월 29일 12:54 오후
e-book 판매가 : 8,000원 (구매하기)
7676
{'title': '점프 투 파이썬', 'author': '박응용', 'edit_time': '최종 편집일시 : 2026년 1월 29일 12:54 오후', 'price': 'e-book 판매가 : 8,000원 (구매하기)', 'up': 7676, 'overview_str': '이 책은 파이썬이란 언어를 처음 접해보는 독자들과 프로그래밍을 한 번도 해 본적이 없는 사람들을 대상으로 한다. 프로그래밍을 할 때 사용되는 전문적인 용어들을 알기 쉽게 풀어서 쓰려고 노력하였으며, 파이썬이란 언어의 개별적인 특성만을 강조하지 않고 프로그래밍 전반에 관한 사항을 파이썬이란 언어를 통해 알 수 있도록 알기 쉽게 설명하였다.\n파이썬에 대한 기본적인 지식을 알고 있는 사람이라도 이 책은 파이썬 프로그래밍에 대한 흥미를 가질 수 있는 좋은 안내서가 될 것이다. 이 책의 목표는 독자가 파이썬을 통해 프로그래밍에 대한 전반적인 이해를 갖게하는 것이며, 또 파이썬이라는 도구를 이용하여 원하는 프로그램을 쉽고 재미있게 만들 수 있게 하는 것이다.\n점프 투 파이썬 개정 2판이 종이책으로 출간되었습니다. (2023년 6월 15일)\n'}
1 번째 책 '점프 투 파이썬' 내용이 수집되었습니다 

딥 러닝을 이용한 자연어 처리 입문
브라이스 유
최종 편집일시 : 2026년 1월 9일 5:09 오후
e-book 판매가 : 33,000원 (구매하기)
6785
{'title': '딥 러닝을 이용한 자연어 처리 입문', 'author': '브라이스 유', 'edit_time': '최종 편집일시 : 2026년 1월 9일 5:09 오후', 'price': 'e-book 판매가 : 33,000원 (구매하기)', 'up': 6785, 'overview_str': '26년 1월 기준: 누적 조회수: 1,950만 돌파 베스트셀러\n많은 분들의 피드백으로 수 년간 보완된 현

In [39]:
df_final = pd.DataFrame.from_dict(total_dict, 'index')
df_final

,title,author,edit_time,price,up,overview_str
0,점프 투 파이썬,박응용,최종 편집일시 : 2026년 1월 29일 12:54 오후,"e-book 판매가 : 8,000원 (구매하기)",7676,이 책은 파이썬이란 언어를 처음 접해보는 독자들과 프로그래밍을 한 번도 해 본적이 ...
1,딥 러닝을 이용한 자연어 처리 입문,브라이스 유,최종 편집일시 : 2026년 1월 9일 5:09 오후,"e-book 판매가 : 33,000원 (구매하기)",6785,"26년 1월 기준: 누적 조회수: 1,950만 돌파 베스트셀러\n많은 분들의 피드백..."
2,딥 러닝 파이토치 교과서 - 입문부터 LLM 파인튜닝까지,브라이스 유,최종 편집일시 : 2026년 1월 9일 5:09 오후,"e-book 판매가 : 33,000원 (구매하기)",5353,26년 1월 기준: 누적 조회수: 490만 돌파 베스트셀러\n딥 러닝 프레임워크 P...
3,파이썬으로 배우는 알고리즘 트레이딩 (개정판-2쇄),조대표,최종 편집일시 : 2023년 5월 30일 1:48 오전,0,4794,파이스탁 YouTube\n리디북스 Ebook 구매 바로가기\n
4,점프 투 자바,박응용,최종 편집일시 : 2026년 1월 16일 2:38 오후,"e-book 판매가 : 8,000원 (구매하기)",4656,이 책은 프로그래머를 꿈꾸며 자바 입문서를 찾는 사람들을 위한 책이다. 이 책은 자...
